<a href="https://colab.research.google.com/github/badstyle-goodstyle/data_engineering/blob/data_engine/data_engineering4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Упражнение:

## Создайте таблицы с указанными столбцами и заполните их произвольными данными.

1. Books (id, author, title, publish_year).
2. Readers (id, name).
3. Records (reader_id, book_id, taking_date, returning_date).

In [47]:
import sqlite3

CREATE_DATA = """
CREATE TABLE IF NOT EXISTS books (
  id integer PRIMARY KEY,
  author text NOT NULL,
  title text NOT NULL,
  published_year integer NOT NULL
); 

CREATE TABLE IF NOT EXISTS readers (
  id integer PRIMARY KEY,
  name text NOT NULL
);

CREATE TABLE IF NOT EXISTS records (
  reader_id integer NOT NULL,
  book_id integer NOT NULL,
  taking_date text,
  returning_date text,
  FOREIGN KEY (reader_id) REFERENCES readers (id),
  FOREIGN KEY (book_id) REFERENCES books (id)
)
"""

ADD_BOOK = """ 
INSERT INTO books(author, title, published_year)
  VALUES (?, ?, ?);
"""

ADDING_BOOKS = [
    ('Л. Н. Толстой', 'Война и мир', 1867),
    ('Ф. М. Достоевский', 'Преступление и наказание', 1866),
    ('Ф. К. Дик', 'Мы вас построим', 2020)
]

ADD_READER = """ 
INSERT INTO readers (name)
  VALUES (?);
"""

ADDING_READERS = [
    ('Шка Иванов',),
    ('Фёдор Лысый',)
]

ADD_RECORD = """ 
INSERT INTO records(reader_id, book_id)
  VALUES (?, ?);
"""

reader_id = 1
book_id = 1
date_to_give_a_book = '2022-12-06'
date_to_take_a_book = '2022-12-31'

ALTER_RECORD_GIVING = """
UPDATE records
  SET taking_date = ?
    WHERE reader_id = ? AND book_id = ?;
"""

ALTER_RECORD_RETURNING = """
UPDATE records
  SET returning_date = ?
    WHERE reader_id = ? AND book_id = ?;
"""

RECORD_GIVING_DATA = (date_to_give_a_book, reader_id, book_id)
RECORD_RETURNING_DATA = (date_to_take_a_book, reader_id, book_id)

ADDING_ONE_RECORD = (reader_id, book_id)

ids_to_delete = [_ for _ in range(10)]

CLEAR_BOOK_SHELF = """
DELETE FROM books WHERE id IN {sql_list}
""".format(sql_list=tuple(ids_to_delete))

CLEAR_READER_LIST = """
DELETE FROM readers WHERE id IN {sql_list}
""".format(sql_list=tuple(ids_to_delete))

CLEAR_RECORD_LIST = """
DELETE FROM records WHERE reader_id IN {sql_list}
""".format(sql_list=tuple(ids_to_delete))

SELECT_ALL_BOOKS = """
SELECT * FROM books;
"""

SELECT_ALL_READERS = """
SELECT * FROM readers;
"""

SELECT_ALL_RECORDS = """
SELECT * FROM records;
"""

In [48]:
connection = sqlite3.connect("my_data.db")
cursor = connection.cursor()

cursor.executescript(CREATE_DATA)
connection.close()

# Задача №1:

## Вывести список книг.

In [49]:
connection = sqlite3.connect("my_data.db")
cursor = connection.cursor()

cursor.execute(CLEAR_BOOK_SHELF)
cursor.executemany(ADD_BOOK, ADDING_BOOKS)
connection.commit()

data = cursor.execute(SELECT_ALL_BOOKS).fetchall()
print(data)

connection.close()

[(1, 'Л. Н. Толстой', 'Война и мир', 1867), (2, 'Ф. М. Достоевский', 'Преступление и наказание', 1866), (3, 'Ф. К. Дик', 'Мы вас построим', 2020)]


# Задача №2:

## Вывести список читателей.

In [50]:
connection = sqlite3.connect("my_data.db")
cursor = connection.cursor()

cursor.execute(CLEAR_READER_LIST)
cursor.executemany(ADD_READER, ADDING_READERS)
connection.commit()

data = cursor.execute(SELECT_ALL_READERS).fetchall()
print(data)

connection.close()


[(1, 'Шка Иванов'), (2, 'Фёдор Лысый')]


# Задача №3:

## Добавить книгу.

In [53]:
connection = sqlite3.connect("my_data.db")
cursor = connection.cursor()

NEW_author = 'В. Г. Сорокин'
NEW_title = 'Сахарный Кремль'
NEW_published_year = 2008

DELETE_EXISTED_BOOK = """
DELETE FROM books WHERE author = '{new_author}' AND title = '{new_title}';
""".format(new_author=NEW_author, new_title=NEW_title)

cursor.execute(DELETE_EXISTED_BOOK)

NEW_BOOK = (NEW_author, NEW_title, NEW_published_year)
cursor.execute(ADD_BOOK, NEW_BOOK)
connection.commit()

data = cursor.execute(SELECT_ALL_BOOKS).fetchall()
for row in data:
  print(row)

connection.close()

(1, 'Л. Н. Толстой', 'Война и мир', 1867)
(2, 'Ф. М. Достоевский', 'Преступление и наказание', 1866)
(3, 'Ф. К. Дик', 'Мы вас построим', 2020)
(4, 'В. Г. Сорокин', 'Сахарный Кремль', 2008)


# Задача №4:

## Добавить книгу.

In [59]:
connection = sqlite3.connect("my_data.db")
cursor = connection.cursor()

NEW_READER = ('Андрей Данилович Комяга',)

DELETE_EXISTED_READER = """
DELETE FROM readers WHERE name = '{new_reader}';
""".format(new_reader=NEW_READER[0])

cursor.execute(DELETE_EXISTED_READER)

cursor.execute(ADD_READER, NEW_READER)
connection.commit()

data = cursor.execute(SELECT_ALL_READERS).fetchall()
print(data)

connection.close()

[(1, 'Шка Иванов'), (2, 'Фёдор Лысый'), (3, 'Андрей Данилович Комяга')]


# Задачи №5 и №6:

## а) выдать книгу читателю;    б) принять книгу.

In [63]:
connection = sqlite3.connect("my_data.db")
cursor = connection.cursor()

cursor.execute(CLEAR_RECORD_LIST)
cursor.execute(ADD_RECORD, ADDING_ONE_RECORD)
connection.commit()

data = cursor.execute(SELECT_ALL_RECORDS).fetchall()
print(f'1. После создания записи: {data}')

# а:
cursor.execute(ALTER_RECORD_GIVING, RECORD_GIVING_DATA)
connection.commit()

data = cursor.execute(SELECT_ALL_RECORDS).fetchall()
print(f'2. После добавления времени отдачи книги в записи: {data}')

# б:
cursor.execute(ALTER_RECORD_RETURNING, RECORD_RETURNING_DATA)
connection.commit()

data = cursor.execute(SELECT_ALL_RECORDS).fetchall()
print(f'3. После добавления времени возврата книги в записи: {data}')

connection.close()

1. После создания записи: [(1, 1, None, None)]
2. После добавления времени отдачи книги в записи: [(1, 1, '2022-12-06', None)]
3. После добавления времени возврата книги в записи: [(1, 1, '2022-12-06', '2022-12-31')]
